In [1]:
%config IPCompleter.greedy=True

In [2]:
from unityagents import UnityEnvironment
import numpy as np
from ddpg_agent import Agent
from collections import deque
from workspace_utils import active_session

In [3]:
#env = UnityEnvironment(file_name='Reacher/Reacher.exe')
env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64')

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [4]:
agent = Agent(state_size=33, action_size=4, random_seed=15)

def ddpg(nEpisodes=2000, maxTimeSteps=1000, print_every=100):
    
    num_agents=20
    scores_deque = deque(maxlen=print_every)
    returnedScores = []
    multiAgentPerEpisodeScores = []
    
    for i_episode in range(1, nEpisodes+1):
        
        env_info = env.reset(train_mode=True)[brain_name]      # reset the environment
        states = env_info.vector_observations                  # get the current state (for each agent)
        
        multiAgentPerEpisodeScores = np.zeros(num_agents)       # initialize the score (for each agent)
        for step in range(maxTimeSteps):
            actions = agent.act(states)
            actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
            
            env_info = env.step(actions)[brain_name]           # send all actions to tne environment
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            
            agent.step(states, actions, rewards, next_states, dones)

            multiAgentPerEpisodeScores += env_info.rewards      # update the score (for each agent)
            states = next_states                               # roll over states to next time step

            if np.any(dones):                                  # exit loop if episode finished
                break
        returnedScores.append(np.mean(multiAgentPerEpisodeScores))
        
        scores_deque.append(np.mean(multiAgentPerEpisodeScores))
        print('\rEpisode {}\tTotal score (averaged over agents): {:.2f}'.format(i_episode, np.mean(scores_deque)), end="")
        if i_episode % print_every == 0:
            print('\rEpisode {}\tTotal score (averaged over agents): {:.2f}'.format(i_episode, np.mean(scores_deque)))
        if np.mean(scores_deque) > 33:
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            print('\rEnvironment solved in {} episodes'.format(i_episode))
            break
    return scores


with active_session():

    returnedScores = ddpg()

    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(np.arange(1, len(returnedScores)+1), returnedScores)
    plt.ylabel('Score')
    plt.xlabel('Episode #')
    plt.show()

Episode 48	Total score (averaged over agents): 0.79

KeyboardInterrupt: 

In [ ]:
env.close()